# Study Systematics Flip

The purpose of this notebook is to check if systematics can be implemented as part of a vector of events in a way that does not increase memory usage.

```Muon_pt_1```, ```Muon_pt_2```, ```Electron_pt_1```, ```Electron_pt_2``` are the variables that also have systematics counterparts.

```PV_x```, ```PV_y```, ```PV_z``` do not have systematics counterparts.

In [1]:
import uproot4
import pickle
import pandas as pd
import numpy as np

In [2]:
# Open root file and ttree

file = uproot4.open('test_sys_signal.root')
tree = file['Events']
tree.show()

name                 | typename                 | interpretation                
---------------------+--------------------------+-------------------------------
Muon_pt_1            | float                    | AsDtype('>f4')
Muon_pt_2            | float                    | AsDtype('>f4')
Electron_pt_1        | float                    | AsDtype('>f4')
Electron_pt_2        | float                    | AsDtype('>f4')
Muon_pt_1_Up         | float                    | AsDtype('>f4')
Muon_pt_2_Up         | float                    | AsDtype('>f4')
Electron_pt_1_Up     | float                    | AsDtype('>f4')
Electron_pt_2_Up     | float                    | AsDtype('>f4')
Muon_pt_1_Down       | float                    | AsDtype('>f4')
Muon_pt_2_Down       | float                    | AsDtype('>f4')
Electron_pt_1_Down   | float                    | AsDtype('>f4')
Electron_pt_2_Down   | float                    | AsDtype('>f4')
PV_x                 | float                    | AsDtype(

In [3]:
nominals_with_sys = ['Muon_pt_1', 'Muon_pt_2', 'Electron_pt_1', 'Electron_pt_2']
nominals_without_sys = ['PV_x', 'PV_y', 'PV_z']
systematics = ['', '_Up', '_Down']

## Numpy

In [29]:
branches_pd = tree.arrays(library='pd')

In [30]:
branches_pd

,Muon_pt_1,Muon_pt_2,Electron_pt_1,Electron_pt_2,Muon_pt_1_Up,Muon_pt_2_Up,Electron_pt_1_Up,Electron_pt_2_Up,Muon_pt_1_Down,Muon_pt_2_Down,Electron_pt_1_Down,Electron_pt_2_Down,PV_x,PV_y,PV_z
0,17.382437,13.238400,44.744175,58.942070,17.882437,13.738400,45.244175,59.442070,16.882437,12.738400,44.244175,58.442070,0.242114,0.395116,6.090504
1,50.051613,41.024353,15.959146,5.622648,50.551613,41.524353,16.459146,6.122648,49.551613,40.524353,15.459146,5.122648,0.240533,0.393753,-6.544194
2,24.702271,41.462658,5.042018,7.994947,25.202271,41.962658,5.542018,8.494947,24.202271,40.962658,4.542018,7.494947,0.245655,0.392481,-12.015012
3,55.392330,32.569691,16.479095,14.080798,55.892330,33.069691,16.979095,14.580798,54.892330,32.069691,15.979095,13.580798,0.246329,0.394249,-4.695963
4,37.216637,54.203739,14.535382,17.012287,37.716637,54.703739,15.035382,17.512287,36.716637,53.703739,14.035382,16.512287,0.245821,0.396207,4.705774
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22833,15.948185,10.853278,34.428360,56.252823,16.448185,11.353278,34.928360,56.752823,15.448185,10.353278,33.928360,55.752823,0.245845,0.393200,-0.096518
22834,3.165293,7.369295,27.072086,28.327864,3.665293,7.869295,27.572086,28.827864,2.665293,6.869295,26.572086,27.827864,0.241983,0.393408,-1.978680
22835,33.588539,35.164211,14.531511,12.232145,34.088539,35.664211,15.031511,12.732145,33.088539,34.664211,14.031511,11.732145,0.247215,0.393896,-1.357113
22836,59.003090,21.311920,5.186389,14.721363,59.503090,21.811920,5.686389,15.221363,58.503090,20.811920,4.686389,14.221363,0.245855,0.390907,-14.366134


In [34]:
np.array(branches_pd['Muon_pt_1'])

array([17.382437, 50.051613, 24.70227 , ..., 33.58854 , 59.00309 ,
       42.957253], dtype=float32)

In [35]:
# Make structured array for Muon_pt_1
muon_pt_1 = np.array([(branches_pd.loc[i, 'Muon_pt_1'], branches_pd.loc[i, 'Muon_pt_1_Up'], branches_pd.loc[i, 'Muon_pt_1_Down']) for i in range(len(branches_pd))], 
                     dtype=[('nominal', 'f4'), ('up', 'f4'), ('down', 'f4')])

In [38]:
muon_pt_1.shape

(22838,)

In [106]:
'''
Attempt: make structured arrays for variables with systematics and flat for the others
Unfortunately, fails when concatenating the two arrays for syst and not-syst
The only way would be https://stackoverflow.com/questions/39754658/concatenate-arrays-with-mixed-types,
but then we loose the possibility to access with arr['nominal'] etc.
'''

df_sys = np.vstack(tuple(np.array([tuple(branches_pd.loc[i, name] for name in list(map(lambda suf: var + suf, systematics))) 
                               for i in range(len(branches_pd))], 
                              dtype=[('nominal', 'f4'), ('up', 'f4'), ('down', 'f4')]) for var in nominals_with_sys)).T

'''
df_nosys = np.vstack(tuple(np.array([((branches_pd.loc[i, var]))
                                     for i in range(len(branches_pd))],
                                    dtype=[('nominal', 'f4')]) for var in nominals_without_sys)).T
'''

df_nosys = np.vstack(tuple(np.array(branches_pd[var]) for var in nominals_without_sys)).T

In [176]:
'''Attempt: df_sys is supposed to be a matrix n_evs x nominals_with_sys where every element is a n_sys-dim array
Does not work becaus when you concatenate the arrays n_evs x n_sys arrays for the different (nominals_with_sys) variables
numpy flattens it
'''
df_sys = np.vstack(tuple(np.array([np.array([branches_pd.loc[i, name] for name in list(map(lambda suf: var + suf, systematics))])
                                   for i in range(len(branches_pd))]) for var in nominals_with_sys))

In [ ]:
np.array([branches_pd.loc[i, name] for name in list(map(lambda suf: 'Muon_pt_1' + suf, systematics))])

In [156]:
a = np.array([np.array([branches_pd.loc[i, name] for name in list(map(lambda suf: 'Muon_pt_1' + suf, systematics))]) for i in range(len(branches_pd))])
b = np.array([np.array([branches_pd.loc[i, name] for name in list(map(lambda suf: 'Electron_pt_1' + suf, systematics))]) for i in range(len(branches_pd))])

In [177]:
'''Attempt: df_sys is a 3D array. It can't be concateneted with a 2D array for df_nosys, unless we fill the gaps with nominals.
This would would probably increase the memory usage.
'''
df_sys = np.array([np.array([np.array([branches_pd.loc[i, name] for name in list(map(lambda suf: var + suf, systematics))])
                                   for i in range(len(branches_pd))]) for var in nominals_with_sys])

In [178]:
df_sys.shape

(4, 22838, 3)

## Attempt: double structured array

The dataframe ends up being a nested structured array. In the outmost one, the fields are the variables (```Muon_pt_1```, ```Electron_pt_1```, ```PV_x```, etc.), whose type is another structured array (```sys_type``` below) for the variables that have systematics, ```float``` for the others. 

Of course, the type would have to be generalized if the types that we retrieve from the tree are not simple floats.

```sys_type``` fields are the systematics themselves, plus ```Nominal```.

In [4]:
import awkward1 as ak

In [5]:
branches = tree.arrays(library='ak')

In [6]:
ak.type(branches)

22838 * {"Muon_pt_1": float32, "Muon_pt_2": float32, "Electron_pt_1": float32, "Electron_pt_2": float32, "Muon_pt_1_Up": float32, "Muon_pt_2_Up": float32, "Electron_pt_1_Up": float32, "Electron_pt_2_Up": float32, "Muon_pt_1_Down": float32, "Muon_pt_2_Down": float32, "Electron_pt_1_Down": float32, "Electron_pt_2_Down": float32, "PV_x": float32, "PV_y": float32, "PV_z": float32}

### Generalize

In [7]:
sys_type = np.dtype([('Nominal', 'float32') if sys == '' else (sys, 'float32') for sys in systematics])

In [8]:
sys_type

dtype([('Nominal', '<f4'), ('_Up', '<f4'), ('_Down', '<f4')])

In [9]:
table_type = np.dtype([(var, sys_type) for var in nominals_with_sys] + [(var, 'f4') for var in nominals_without_sys])

In [10]:
table_type

dtype([('Muon_pt_1', [('Nominal', '<f4'), ('_Up', '<f4'), ('_Down', '<f4')]), ('Muon_pt_2', [('Nominal', '<f4'), ('_Up', '<f4'), ('_Down', '<f4')]), ('Electron_pt_1', [('Nominal', '<f4'), ('_Up', '<f4'), ('_Down', '<f4')]), ('Electron_pt_2', [('Nominal', '<f4'), ('_Up', '<f4'), ('_Down', '<f4')]), ('PV_x', '<f4'), ('PV_y', '<f4'), ('PV_z', '<f4')])

In [11]:
%%time

final = np.array([tuple([np.array(tuple([branches[name][i] for name in list(map(lambda suf: var + suf, systematics))]), sys_type) for var in nominals_with_sys] 
                        + [branches[var][i] for var in nominals_without_sys]) 
                  for i in range(len(branches))], dtype=table_type)

CPU times: user 9.33 s, sys: 86.5 ms, total: 9.42 s
Wall time: 9.42 s


### Try some operations

In [12]:
final.shape

(22838,)

In [13]:
# Slicing

for i, evt in enumerate(final[0:3]):
    print('Event {}: {}'.format(i, evt))

Event 0: ((17.382437, 17.882437, 16.882437), (13.2383995, 13.7383995, 12.7383995), (44.744175, 45.244175, 44.244175), (58.94207, 59.44207, 58.44207), 0.24211372, 0.39511582, 6.090504)
Event 1: ((50.051613, 50.551613, 49.551613), (41.024353, 41.524353, 40.524353), (15.9591465, 16.459146, 15.4591465), (5.622648, 6.122648, 5.122648), 0.24053279, 0.39375338, -6.5441937)
Event 2: ((24.70227, 25.20227, 24.20227), (41.462658, 41.962658, 40.962658), (5.042018, 5.542018, 4.542018), (7.9949474, 8.494947, 7.4949474), 0.24565527, 0.39248142, -12.015012)


In [14]:
final['Muon_pt_1']

array([(17.382437, 17.882437, 16.882437),
       (50.051613, 50.551613, 49.551613),
       (24.70227 , 25.20227 , 24.20227 ), ...,
       (33.58854 , 34.08854 , 33.08854 ),
       (59.00309 , 59.50309 , 58.50309 ),
       (42.957253, 43.457253, 42.457253)],
      dtype=[('Nominal', '<f4'), ('_Up', '<f4'), ('_Down', '<f4')])

In [15]:
for i, evt in enumerate(final['Muon_pt_1'][0:3]):
    print('Event {}: Nominal {}, Up {}, Down {}'.format(i, evt['Nominal'], evt['_Up'], evt['_Down']))

Event 0: Nominal 17.382436752319336, Up 17.882436752319336, Down 16.882436752319336
Event 1: Nominal 50.051612854003906, Up 50.551612854003906, Down 49.551612854003906
Event 2: Nominal 24.7022705078125, Up 25.2022705078125, Down 24.2022705078125


### Taggers

In [16]:
import pickle

In [17]:
# unpickle trained model
bdt = pickle.load(open('classifier.pkl', 'rb'))

First we study how to add a new column to a structured Numpy array

In [41]:
new_col = np.array([tuple(np.array([(0,0,0)], dtype=sys_type)) for i in range(len(final))], dtype=[('Predicted', sys_type)])

In [45]:
new_col['Predicted']['Nominal']

array([0., 0., 0., ..., 0., 0., 0.], dtype=float32)

In [33]:
import numpy.lib.recfunctions as rfn

In [ ]:
#final_added = rfn.merge_arrays((final, new_col), flatten=True)

In [54]:
# Making a whole new one

new_dtype = np.dtype(final.dtype.descr + [('Predicted', sys_type)])

In [57]:
new_df = np.empty(final.shape, dtype=new_dtype)

In [58]:
for var in nominals_with_sys + nominals_without_sys:
    new_df[var] = final[var]
new_df['Predicted'] = new_col['Predicted']

In [71]:
new_df

array([((17.382437, 17.882437, 16.882437), (13.2383995, 13.7383995, 12.7383995), (44.744175 , 45.244175 , 44.244175 ), (58.94207  , 59.44207  , 58.44207  ), 0.24211372, 0.39511582,   6.090504 , (0., 0., 0.)),
       ((50.051613, 50.551613, 49.551613), (41.024353 , 41.524353 , 40.524353 ), (15.9591465, 16.459146 , 15.4591465), ( 5.622648 ,  6.122648 ,  5.122648 ), 0.24053279, 0.39375338,  -6.5441937, (0., 0., 0.)),
       ((24.70227 , 25.20227 , 24.20227 ), (41.462658 , 41.962658 , 40.962658 ), ( 5.042018 ,  5.542018 ,  4.542018 ), ( 7.9949474,  8.494947 ,  7.4949474), 0.24565527, 0.39248142, -12.015012 , (0., 0., 0.)),
       ...,
       ((33.58854 , 34.08854 , 33.08854 ), (35.16421  , 35.66421  , 34.66421  ), (14.531511 , 15.031511 , 14.031511 ), (12.232145 , 12.732145 , 11.732145 ), 0.24721472, 0.3938958 ,  -1.3571125, (0., 0., 0.)),
       ((59.00309 , 59.50309 , 58.50309 ), (21.31192  , 21.81192  , 20.81192  ), ( 5.1863894,  5.6863894,  4.6863894), (14.721363 , 15.221363 , 14.22136

In [67]:
def perform_inference(df, clf, nominals, systematics, new_column):
    '''Given a classifier that takes n columns as input, recursively apply
    the classifier on the n columns specified by the combination of nominal
    with the elements in systematics
    '''
    model_features = clf.get_booster().feature_names
    for sys in systematics:
        columns = list(map(lambda pref: pref + sys, nominals))
        df[new_column + sys] = clf.predict(df.rename(
            columns=dict(zip(columns, model_features)), inplace=False)[model_features])
        df = df.rename(columns=dict(zip(model_features, columns)))
    return df

In [65]:
# First we try to produce a new_column with the results of the application of the booster to final
p = pd.DataFrame(final)
df = perform_inference(p, bdt, nominals_with_sys, systematics, new_column='Y')

In [80]:
# Create 2D array from final in order to feed the classifier
inp = np.array([[final[var][i] for var in nominals_with_sys] for i in range(len(final))])

In [82]:
inp[:, 0]

array([(17.382437, 17.882437, 16.882437),
       (50.051613, 50.551613, 49.551613),
       (24.70227 , 25.20227 , 24.20227 ), ...,
       (33.58854 , 34.08854 , 33.08854 ),
       (59.00309 , 59.50309 , 58.50309 ),
       (42.957253, 43.457253, 42.457253)],
      dtype=[('Nominal', '<f4'), ('_Up', '<f4'), ('_Down', '<f4')])

# Awkward

In [194]:
import awkward1 as ak

In [209]:
branches = tree.arrays(library='ak')

In [241]:
branches

<Array [{Muon_pt_1: 17.4, ... PV_z: 7.01}] type='22838 * {"Muon_pt_1": float32, ...'>

In [242]:
ak.type(branches)

22838 * {"Muon_pt_1": float32, "Muon_pt_2": float32, "Electron_pt_1": float32, "Electron_pt_2": float32, "Muon_pt_1_Up": float32, "Muon_pt_2_Up": float32, "Electron_pt_1_Up": float32, "Electron_pt_2_Up": float32, "Muon_pt_1_Down": float32, "Muon_pt_2_Down": float32, "Electron_pt_1_Down": float32, "Electron_pt_2_Down": float32, "PV_x": float32, "PV_y": float32, "PV_z": float32}

In [218]:
len(branches)

22838

### Step by step

In [213]:
[branches[name][0] for name in list(map(lambda suf: 'Muon_pt_1'+suf, systematics))]

[17.382436752319336, 17.882436752319336, 16.882436752319336]

In [215]:
ak.Array([branches[name][0] for name in list(map(lambda suf: 'Muon_pt_1' + suf, systematics))])

<Array [17.4, 17.9, 16.9] type='3 * float64'>

In [219]:
ak.Array([ak.Array([branches[name][0] for name in list(map(lambda suf: 'Muon_pt_1' + suf, systematics))]) for i in range(len(branches))])

<Array [[17.4, 17.9, 16.9, ... 17.9, 16.9]] type='22838 * var * float64'>

In [221]:
np_array = np.array([ak.Array([ak.Array([branches[name][0] for name in list(map(lambda suf: 'Muon_pt_1' + suf, systematics))]) for i in range(len(branches))])], dtype=[('nominal', 'f4'), ('up', 'f4'), ('down', 'f4')])

In [223]:
ak.type(ak.from_numpy(np_array))

1 * {"nominal": 22838 * 3 * float32, "up": 22838 * 3 * float32, "down": 22838 * 3 * float32}

In [243]:
muon_pt_numpy = np.array([tuple(branches[name][i] for name in list(map(lambda suf: 'Muon_pt_1' + suf, systematics))) 
                               for i in range(len(branches))], 
                              dtype=[('nominal', 'f4'), ('up', 'f4'), ('down', 'f4')])

In [244]:
muon_pt_numpy

array([(17.382437, 17.882437, 16.882437),
       (50.051613, 50.551613, 49.551613),
       (24.70227 , 25.20227 , 24.20227 ), ...,
       (33.58854 , 34.08854 , 33.08854 ),
       (59.00309 , 59.50309 , 58.50309 ),
       (42.957253, 43.457253, 42.457253)],
      dtype=[('nominal', '<f4'), ('up', '<f4'), ('down', '<f4')])

In [245]:
type(muon_pt_numpy)

numpy.ndarray

In [254]:
muon_pt_numpy.shape

(22838,)

In [246]:
electron_pt_numpy = np.array([tuple(branches[name][i] for name in list(map(lambda suf: 'Electron_pt_1' + suf, systematics))) 
                               for i in range(len(branches))], 
                              dtype=[('nominal', 'f4'), ('up', 'f4'), ('down', 'f4')])

In [247]:
electron_pt_numpy

array([(44.744175 , 45.244175 , 44.244175 ),
       (15.9591465, 16.459146 , 15.4591465),
       ( 5.042018 ,  5.542018 ,  4.542018 ), ...,
       (14.531511 , 15.031511 , 14.031511 ),
       ( 5.1863894,  5.6863894,  4.6863894),
       ( 6.6560383,  7.1560383,  6.1560383)],
      dtype=[('nominal', '<f4'), ('up', '<f4'), ('down', '<f4')])

In [248]:
type(electron_pt_numpy)

numpy.ndarray

In [249]:
pv_numpy = np.array(branches['PV_x'])

In [250]:
pv_numpy

array([0.24211372, 0.24053279, 0.24565527, ..., 0.24721472, 0.24585512,
       0.24327561], dtype=float32)

In [251]:
type(pv_numpy)

numpy.ndarray

In [253]:
pv_numpy.shape

(22838,)

In [265]:
tp = np.dtype([('nominal', 'f4'), ('up', 'f4'), ('down', 'f4')]) 

In [ ]:
all_numpy = np.array()